In [1]:
from src.responsabilisation_classes import responsabilisation,deresponsabilisation,neutre

import ollama
import pandas as pd

from datetime import datetime

import time


In [4]:
telma = pd.read_excel('responsabilisation_test_sample_telma.xlsx')
robin = pd.read_excel('responsabilisation_test_sample_robin.xlsx')

In [7]:
robin.columns

Index(['text', 'robin_pred', 'source', 'llm_output_two', 'llm_pred_two'], dtype='object')

In [14]:
def encode_label(label):
    if "déresponsabilisation" in label:
        return 2
    elif "responsabilisation" in label:
        return 1
    else:
        return 0

telma['telma_pred'] = telma['label_telma'].apply(lambda x: encode_label(x.lower() if type(x) == str else "neutre"))

In [25]:
data = robin[:100].merge(telma[:100], on='text',).dropna(subset=['robin_pred'])

In [26]:
len(data)

99

In [33]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report

acc = accuracy_score(data['robin_pred'],data['telma_pred'])
f1 = f1_score(data['telma_pred'], data['robin_pred'], average='weighted')
precision = precision_score(data['telma_pred'], data['robin_pred'], average='weighted')
recall = recall_score(data['telma_pred'], data['robin_pred'], average='weighted')
specificity = recall_score(data['telma_pred'], data['robin_pred'], average='weighted', pos_label=0)

print(f"Accuracy: {acc}")
print(f"F1: {f1}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"Specificity: {specificity}")

print(classification_report(data['telma_pred'], data['robin_pred'], target_names=['neutre', 'responsabilisation', 'deresponsabilisation']))

Accuracy: 0.5454545454545454
F1: 0.5561403932518232
Precision: 0.5944719682964197
Recall: 0.5454545454545454
Specificity: 0.5454545454545454
                      precision    recall  f1-score   support

              neutre       0.61      0.66      0.64        41
  responsabilisation       0.27      0.47      0.34        15
deresponsabilisation       0.69      0.47      0.56        43

            accuracy                           0.55        99
           macro avg       0.52      0.53      0.51        99
        weighted avg       0.59      0.55      0.56        99



/home/robin/env/violent_men/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1523: UserWarning: Note that pos_label (set to 0) is ignored when average != 'binary' (got 'weighted'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(


In [32]:
classification_report(data['telma_pred'], data['robin_pred'], target_names=['neutre', 'responsabilisation', 'deresponsabilisation'])

'                      precision    recall  f1-score   support\n\n              neutre       0.61      0.66      0.64        41\n  responsabilisation       0.27      0.47      0.34        15\nderesponsabilisation       0.69      0.47      0.56        43\n\n            accuracy                           0.55        99\n           macro avg       0.52      0.53      0.51        99\n        weighted avg       0.59      0.55      0.56        99\n'

In [2]:
instruction_system = f""" 

You are a psychiatrist with expertise in violent men. 
You have received excerpts from testimonies and you have to judge whether the excerpt is neutral or associated with responsibility for the violence.

You have 3 cases to distinguish:
- Neutral: the excerpt is not about violence or responsibility for violence.
- Responsibilization: The person takes responsibility for mistakes and violence.
- De-responsibilization: The person denies responsibility for mistakes and violence. They usually try to find excuses or present themselves as victims.
###
"""


#neural-chat:7b
def from_testimony_to_template_1_example(testimony):
    
    template = f"""
 
        
        [INST]
        Select between the tree classes:
        - De-responsibilization
        - Neutral
        - Responsibilization
        

        You should ask yourself the following questions:
        - Is the excerpt about violence? if no then this is neutral.
        - is the person talking about couple relationship? if no then this is neutral.
        - Is the person taking responsibility for the violence? if yes then this is responsibilization.
        - is the person acknowledging their mistakes? if yes then this is responsibilization.
        - Is the person trying to find excuses or presenting themselves as a victim? if yes then this is de-responsibilization.
  

        
        [\INST]

        Here are examples : 
        excerpt : "Je savais pas ce qu’est-ce que j’allais faire je suis rentré dans l’armée j’étais un enfant"
        answer: #Neutral
        excerpt : "moi j’ai pas fait j’ai jamais rendu jalouse mais elle oui pis elle entretenait le pattern pis elle voulait dont pas reconnaitre certaines situations" 
        answer: #De-responsibilization
        excerpt : "dans le cadre conjugal j’ai j’ai jamais touché à une de mes conjointes physiquement c’est surtout verbalement psychologiquement"
        answer: #Responsibilization
        excerpt : ""ça j’en j’en étais pas conscient parce qu’il y avait pas de coup physique",
        answer: #Responsibilization

        Put your answer in the following format: #answer.
        <<<
        here is the excerpt: {testimony} 
        >>>

        """
    
    return template

def from_testimony_to_template_2_example(testimony):
    
    template = f"""
 
        
        [INST]
        Select between the tree classes:
        - Neutral
        - Responsibilization
        - De-responsibilization

        You should ask yourself the following questions:
        - Is the excerpt about violence? if no then this is neutral.
        - is the person talking about couple relationship? if no then this is neutral.
        - Is the person taking responsibility for the violence? if yes then this is responsibilization.
        - Is the person regretting their actions? if yes then this is responsibilization.
        - is the person acknowledging their mistakes? if yes then this is responsibilization.
        - Is the person trying to find excuses? if yes then this is de-responsibilization.
        - Is the person presenting themselves as a victim? if yes then this is de-responsibilization.
        - Is the person accusing someone else? if yes then this is de-responsibilization.
        
        [\INST]

        Here are examples : 
        excerpt : "Je savais pas ce qu’est-ce que j’allais faire je suis rentré dans l’armée j’étais un enfant" 
        answer: #Neutral
        excerpt : "moi j’ai pas fait j’ai jamais rendu jalouse mais elle oui pis elle entretenait le pattern pis elle voulait dont pas reconnaitre certaines situations" 
        answer: #De-responsibilization
        excerpt : "dans le cadre conjugal j’ai j’ai jamais touché à une de mes conjointes physiquement c’est surtout verbalement psychologiquement" 
        answer: #Responsibilization
        excerpt : "je le sais que je suis impulsif si on m’attaque verbalement j’attaque verbalement c’est c’est c’est automatique essayer de contourner cette mauvaise habitude là ce mauvais comportement là" 
        answer: #Reponsibilization
        excerpt : "quand il est parti en ambulance il avait un pouls mais ils ont pas été en mesure de le réanimer" answer: #Neutral
        excerpt : "is non je m’en rends pas toujours compte parce que j’ai une oreille à 50% de capacité je me rends pas compte quand je commence à parler hyper énervé j’ai le ton de voix qui monte je cri pas mais j’ai le ton de voix qui monte pis j’ai déjà une bonne voix portante faque mais sinon j’ai j’en ai pas de frapper menacer" 
        answer: #De-responsibilization
        excerpt : "il faut qu’il y ait des projets communs une vie commune une complicité aimer les mêmes affaires mais pas tout à fait les mêmes affaires en tout cas au moins partager certaines choses" 
        answer: #Neutral
        excerpt: "j’ai fait de la violence verbale psychologique parce que j’étais un peu manipulateur aussi"
        answer: #Responsibilization
        excerpt: "moi aussi je pourrais être agressif tu sais pis je lui ai dit puis j’ai brandi le poing devant elle tu sais je pourrais faire ça comme ça pis crier mais je le fais pas tu sais mais en le faisant ça ça l’a traumatisé elle ça l’a marqué tu sais faque j’aurais pas dû faire ça mais je l’ai jamais touché je l’ai jamais frappé tu sais donc EUH"
        answer: #Responsibilization
        
        Put your answer in the following format: #answer.
        <<<
        here is the excerpt: {testimony} 
        >>>

        """
    
    return template

def from_testimony_to_template_zero_example(testimony):
    
    template = f"""
 
        
        [INST]
        Select between the tree classes:
        - Neutral
        - Responsibilization
        - De-responsibilization

        You should ask yourself the following questions:
        - Is the excerpt about violence?
        - Is the person taking responsibility for the violence?
        - Is the person trying to find excuses?
        - Is the person presenting themselves as a victim?
        
        Put your answer in the following format: #answer.
        
        <<<
        here is the excerpt: {testimony} 
        >>>

        """
    
    return template

In [3]:
data = pd.DataFrame()
data['text'] = responsabilisation + deresponsabilisation + neutre
data['label'] = ['responsabilisation']*len(responsabilisation) + ['deresponsabilisation']*len(deresponsabilisation) + ['neutre']*len(neutre)
data = data.sample(frac=1).reset_index(drop=True)

In [4]:
def compute_llm_analysis(text, label, model = 'neural-chat:7b',example = 1,start_time = None):
    print("##" * 50)
    if example == 0:
        template = from_testimony_to_template_zero_example(text)
    elif example == 1:
        template = from_testimony_to_template_1_example(text)
    elif example == 2:
        template = from_testimony_to_template_2_example(text)
    else : 
        raise ValueError("example must be 0 or 1")
    r = ollama.generate(model=model, prompt=template,  options = {"temperature": 0, "seed":42}, keep_alive = "10s", system = instruction_system, context = None)
    print("## True answer: ", label)
    print(f"## {r['response']}")
    if start_time:
        if (datetime.now() - start_time).seconds % 120 <= 5: # every 2 minutes we stop for 30 seconds
            print("sleep 45 " * 50)
            time.sleep(45)
        
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")
    print("Current Time =", current_time)
    print("##" * 50)
    #time.sleep(15)
    return r['response']


#data['llm_output_one'] = data.apply(lambda x: compute_llm_analysis(x['text'], x['label'],example=1), axis = 1)
#time.sleep(40)
#data['llm_output_two'] = data.apply(lambda x: compute_llm_analysis(x['text'], x['label'],example=2), axis = 1)
#time.sleep(40)
#data['llm_output_zero'] = data.apply(lambda x: compute_llm_analysis(x['text'], x['label'],example=0), axis = 1)



In [ ]:
def norm_llm_output(x):
    if "#Neutral" in x:
        return 0
    elif "#Responsibilization" in x:
        return 1
    elif "#De-responsibilization" in x:
        return 2
    else:
        return -1

def norm_label(x):
    if x == 'responsabilisation':
        return 1
    elif x == 'deresponsabilisation':
        return 2
    else:
        return 0

data['llm_pred_one'] = data['llm_output_one'].apply(lambda x: norm_llm_output(x))
data['llm_pred_two'] = data['llm_output_two'].apply(lambda x: norm_llm_output(x))
data['llm_pred_zero'] = data['llm_output_zero'].apply(lambda x: norm_llm_output(x))
data['true'] = data['label'].apply(lambda x: norm_label(x))


from sklearn.metrics import accuracy_score, classification_report, f1_score


print(f1_score(data['true'], data['llm_pred_one'], average = 'weighted'))
print(accuracy_score(data['true'], data['llm_pred_one']))
print(classification_report(data['true'], data['llm_pred_one']))

In [6]:
print(f1_score(data['true'], data['llm_pred_zero'], average = 'weighted'))
print(accuracy_score(data['true'], data['llm_pred_zero']))
print(classification_report(data['true'], data['llm_pred_zero']))

0.5393322014714205
0.6129032258064516
              precision    recall  f1-score   support

           0       0.67      0.67      0.67         6
           1       0.50      0.10      0.17        10
           2       0.61      0.93      0.74        15

    accuracy                           0.61        31
   macro avg       0.59      0.57      0.52        31
weighted avg       0.58      0.61      0.54        31



In [7]:
print(f1_score(data['true'], data['llm_pred_one'], average = 'weighted'))
print(accuracy_score(data['true'], data['llm_pred_one']))
print(classification_report(data['true'], data['llm_pred_one']))

0.6228878648233487
0.6774193548387096
              precision    recall  f1-score   support

           0       0.75      1.00      0.86         6
           1       1.00      0.20      0.33        10
           2       0.62      0.87      0.72        15

    accuracy                           0.68        31
   macro avg       0.79      0.69      0.64        31
weighted avg       0.77      0.68      0.62        31



In [8]:
print(f1_score(data['true'], data['llm_pred_two'], average = 'weighted'))
print(accuracy_score(data['true'], data['llm_pred_two']))
print(classification_report(data['true'], data['llm_pred_two']))

0.7194083588770496
0.7419354838709677
              precision    recall  f1-score   support

           0       0.71      0.83      0.77         6
           1       0.80      0.40      0.53        10
           2       0.74      0.93      0.82        15

    accuracy                           0.74        31
   macro avg       0.75      0.72      0.71        31
weighted avg       0.75      0.74      0.72        31



In [6]:
corpus  = pd.read_hdf('/home/robin/Data/Etude_telma/20240227_text_database.h5', key = "df")
corpus.head(2)

,code,duration,interactions,comments,Q1,Q2,Q3,Q4,source,text,text_Q1,text_Q2,text_Q3,text_Q4
0,P34,1620,"[{'speaker': 'experimenter', 'text': ' Q1'}, {...","[Transcription 34 , 27 min , Énervant à retran...","[{'speaker': 'experimenter', 'text': ' Q1'}, {...","[{'speaker': 'experimenter', 'text': ' Q2'}, {...","[{'speaker': 'experimenter', 'text': ' Q3'}, {...","[{'speaker': 'experimenter', 'text': ' Q4'}, {...",france,oui donc pour moi c’est le décès de mon petit...,oui donc pour moi c’est le décès de mon petit...,BAH EUH non parce que je vois pas idées suici...,Alors au niveau du couple je maintiens que ça...,BAH très bien c’est bête j’aurais dû vous l’a...
1,P16,1560,"[{'speaker': 'experimenter', 'text': ' Q1'}, {...",[26 min – 11 bizarres en 15 min ],"[{'speaker': 'experimenter', 'text': ' Q1'}, {...","[{'speaker': 'experimenter', 'text': ' ouais o...","[{'speaker': 'experimenter', 'text': ' ok Q3'}...","[{'speaker': 'experimenter', 'text': ' ok Q4'}...",france,EUH BAH l’é_ l’évènement EUH excusez-moi c’es...,EUH BAH l’é_ l’évènement EUH excusez-moi c’es...,des auto-violences non j’ai jamais fait ça\n ...,et att_ pour le plaisir c’est pour ça que qua...,BAH j’ai pensé que c’était c’était c’était AL...


In [7]:
def flatten_list(l):
    return [item for sublist in l for item in sublist]

def split_text_with_overlap(text,code, k = 75, p = 10):
    words = text.split()
    sequences = []
    for i in range(0, len(words) - k + 1, k - p):
        sequences.append(" ".join(words[i:i + k]))
    return sequences, [code]*len(sequences)


corpus['sequence_list'], corpus['code_list'] = \
zip(*corpus.apply(lambda x :split_text_with_overlap(x['text_Q3'],x.code, k = 75, p = 10),axis=1).tolist())



In [8]:
test = pd.DataFrame()
test['text'] = flatten_list(corpus['sequence_list'].tolist())
test['source'] = flatten_list(corpus['code_list'].tolist())
test_sample = test.sample(n=200).reset_index(drop=True)
len(test_sample), len(test)

(200, 786)

In [ ]:
start_time = datetime.now()
test_sample['llm_output_two'] = test_sample['text'].apply(lambda x: compute_llm_analysis(x, 'unknown',example=2,start_time = start_time))
test_sample['llm_pred_two'] = test_sample['llm_output_two'].apply(lambda x: norm_llm_output(x))

In [11]:
test_sample.to_csv('responsabilisation_test_sample.csv',index = False, sep = "\t")

In [62]:
test['llm_output_norm'] = test['llm_output_two'].apply(norm_llm_output)

In [64]:
test[test['llm_output_norm']==2].sample(10)['text'].to_list()

['je suis quelqu’un je suis quelqu’un qui aime beaucoup le sexe pis elle aussi d’ailleurs elle aussi d’ailleurs mais je voulais tout le temps plus j’avais faim j’en voulait tout le temps pis des bout avant c’est important ce que tu vas entendre avant à la à la elle m’avait',
 'ALL non une fois il s’est reproduit qu’une fois c’est du harcèlement par téléphone j’ai dû aller une fois devant chez elle mais c’est pareil aussi EUH elle avait le le c’est ça c’est que PS c’est c’est c’est con à dire mais EUH par exemple j’allais / on discutait',
 'sur moi c’est comme si PS je sais pas elle était sortie elle est revenue j’ai rien compris j’ai absolument rien compris Non elle s’est jetée sur moi et là elle m’a bousculé jusqu’au ALL à l’intérieur de la de la chambre pour aller me cogner contre les effets là-bas',
 'voulait pas l’entendre pis là après ça elle elle s’est mis comme à je sais pas ce qu’elle me disait mais elle me chia_ elle me chialait après elle me chicanait pis elle elle levait le 